# Session 2

## Contents: 
- First-Class Citezens in programming
  - Higher-Order Functions
  - Closures
  - Pipeline Showcase
- lambda expressions
- Decorators
- Comprehension
- Dunder Methods & the Python Data Model.
- Generators
- Inheritance
- Access modifires in python
  


## First-Class Functions
A programming language is said to have first-class functions if treats functions as _"First-class Citizens"_

### But what is a __*"First-Class Citizen"*__ in programming?
A first-class citizen in a programming language is an entity which supports all the operations generally available to other entities. These operations typically include being passed as an argument, returned from a function, and assigned to a variable.

In [ ]:
# Assigning function to a variable

# A function that returns the square of a given number
def square(x):
    return x*x


f = square
print(square)
print(f(5))


## Higher order functions
A function that takes a function as an argument or returns a function as a result is called a _"higher order function"_.

In [ ]:
# Let's build our own map function!
def my_map(func, iterable):
    result = []
    for element in iterable:
        result.append(func(element))
    return result

# Excercise:
# Todo 1: Move to comprehension exercises
# Apply list comprehesions to the map function that we just wrote.

# def my_map(func, iterable):
#     return [func(i) for i in iterable]

# Let's try it out!
print(my_map(square, [1, 2, 3, 4, 5]))

def cube(num):
    return num ** 3

print(my_map(cube, [1, 2, 3, 4, 5]))



In [ ]:
# Returning a function from another function
def logger(message):
    def log_message():
        return f'Log: {message}'
    return log_message

# Let's say Hi!
log_hi = logger('Hi!')
print(log_hi())

In [ ]:
# HTML tag generator
def html_tag(tag):
    def wrap_text(msg):
        return f'<{tag}>{msg}</{tag}>'
    return wrap_text

h1 = html_tag('h1')
print(h1.__name__)
print(h1('Test Headline'))

print(h1('Another Headline!'))

p = html_tag('p')
print(p('Test Paragraph'))


In [ ]:
# Exercise:
# Todo 2: Write a function that takes a function as an argument and returns a function.
# def operate(func, *args):
#     return func(*args)

# my_function = operate(square, 5)
# print(my_function)
# print(operate(square, 5))

## Closures
A closure is a function that remembers values in enclosing scopes even if they are _not present_ in memory when the function is called.

## Free variables
Variables that can still be refrenced in the body of a function even if it is not present in memory when the function is called.

In [ ]:
# An example of closure
def outter_func():
    # message is a free variable!
    message = 'Hi!'
    def inner_func():
        print(message)
    return inner_func

# func = outter_func()
# print(func.__name__)
# func()

In [ ]:
def another_outter_func(msg):
    message = msg
    def inner_func():
        print(message)
    return inner_func
hi = another_outter_func('Hi!')
bye = another_outter_func('Bye!')
hi()
bye()

### Note:
A closure closes over the free variables in its enclosing scope.

In [36]:
# Pipeline Showcase!

def pipeline_component(func, data, args):
    """A pipeline component which is respnonsible for sending functional arguments over
    to the selected target function.
    """

    return func(data, *args)


def pipeline(data: list, components: list):
    """A pipeline component that helps with making filtering easier. It provides
    access to different filtering mechanism by simplying letting users
    pass in what filter they want to apply, and the arguments for that filter

    Usage::
        >>> # assume variables 'data', 'kwargs'
        >>> pipeline(
            data=data,
            components=[
                {"filter": "square"},
                {"filter": "cube"}
            ]
        )
    """

    # Python treats list, dict, tuple, and set objects as passed reference, thus
    # in order to not modify the previous state, we make a local copy
    if (
        isinstance(data, list)
        or
        isinstance(data, tuple)
        or 
        isinstance(data, set)
        or 
        isinstance(data, dict)
    ):
        __data = data.copy()
    else:
        __data = data
    # A mapping of different filters possible
    function_mappings = {
        "square": square,
        "cube": cube,
        "add": add,
        # Simply add the mapping of a new function,
        # nothing else will really need to changed
    }

    # Going through each of the components
    for component in components:

        # If component is simply empty, continue to next
        # iteration
        if component == {}:
            continue

        # Send to pipeline component, return data to `__data`
        __data = pipeline_component(
            # Map function respectively using the function_mappings dictionary
            func=function_mappings[f'{component["filter"]}'],
            # Send over the data
            data=__data,
            # Except the filter name, select the rest as args
            args=tuple(list(component.values())[1:]),
        )

    # Return the data
    return __data

def add(x, y):
    return x + y

data = pipeline(
    data=2,
    components=[
        {"filter": "square"},
        {"filter": "cube"},
        {"filter": "add", "num": 3},
    ]
)
print(data)


67
